In [3]:
%pip install pandas
import pandas as pd

# Load the CSV content into a DataFrame
df = pd.read_csv('file/bquxjob_157d5c5d_1941367a86e.csv')

# Display the first few rows of the DataFrame to understand its structure
df.head()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: c:\Users\user\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


,Fecha,Campania,Plataforma,inversion,evento_objetivo,Instalaciones,cvr
0,2024-12-14,BOOMIT_PEIG_EC_GADS_ADQUISI_ANDROID_(PRIMTARJ)...,Google Ads,424.01,120,431,0.278422
1,2024-12-14,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(GAMERS)_I...,Meta,31.16,9,27,0.333333
2,2024-12-14,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...,Meta,58.88,19,45,0.422222
3,2024-12-14,BOOMIT_PEIG_EC_META_TRAFICO_GRAL_(BIG-PROMO)_B...,Meta,10.31,0,0,0.000000
4,2024-12-14,BOOMIT_PEIG_EC_META_TRAFICO_GRAL_(PROMO-PEIGO-...,Meta,19.21,0,0,0.000000


In [4]:
# Convert the 'Fecha' column to datetime format
df['Fecha'] = pd.to_datetime(df['Fecha'])

# Identify the most recent date in the dataset
most_recent_date = df['Fecha'].max()

# Define the two periods for analysis
current_period_end = most_recent_date
current_period_start = current_period_end - pd.Timedelta(days=6)
previous_period_end = current_period_start - pd.Timedelta(days=1)
previous_period_start = previous_period_end - pd.Timedelta(days=6)

# Filter data for the two periods
current_period_data = df[(df['Fecha'] >= current_period_start) & (df['Fecha'] <= current_period_end)]
previous_period_data = df[(df['Fecha'] >= previous_period_start) & (df['Fecha'] <= previous_period_end)]

# Display the date ranges for the periods
(current_period_start, current_period_end), (previous_period_start, previous_period_end)

((Timestamp('2024-12-22 00:00:00'), Timestamp('2024-12-28 00:00:00')),
 (Timestamp('2024-12-15 00:00:00'), Timestamp('2024-12-21 00:00:00')))

In [5]:
# Exclude campaigns based on the given criteria
# 1. Campaigns with zero investment
# 2. Campaigns with zero events_objetivo
# 3. Campaigns with less than 3 days of data in either period

# Function to filter campaigns based on the criteria
def filter_campaigns(data, period_start, period_end):
    # Group by campaign and platform
    grouped = data.groupby(['Campania', 'Plataforma'])

    # Calculate the number of days, total investment, and total events_objetivo for each campaign
    summary = grouped.agg(
        days_active=('Fecha', 'nunique'),
        total_investment=('inversion', 'sum'),
        total_events=('evento_objetivo', 'sum')
    ).reset_index()

    # Apply the exclusion criteria
    filtered = summary[
        (summary['total_investment'] > 0) &
        (summary['total_events'] > 0) &
        (summary['days_active'] >= 3)
    ]

    return filtered

# Filter campaigns for both periods
filtered_current_period = filter_campaigns(current_period_data, current_period_start, current_period_end)
filtered_previous_period = filter_campaigns(previous_period_data, previous_period_start, previous_period_end)

# Merge the filtered data from both periods to find campaigns with data in both periods
merged_filtered = pd.merge(
    filtered_current_period,
    filtered_previous_period,
    on=['Campania', 'Plataforma'],
    suffixes=('_current', '_previous')
)

# Calculate cost per objective and percentage variation for each campaign
merged_filtered['costo_por_objetivo_current'] = merged_filtered['total_investment_current'] / merged_filtered['total_events_current']
merged_filtered['costo_por_objetivo_previous'] = merged_filtered['total_investment_previous'] / merged_filtered['total_events_previous']
merged_filtered['variacion_porcentual'] = ((merged_filtered['costo_por_objetivo_current'] - merged_filtered['costo_por_objetivo_previous']) / merged_filtered['costo_por_objetivo_previous']) * 100

# Round the percentage variation to two decimal places
merged_filtered['variacion_porcentual'] = merged_filtered['variacion_porcentual'].round(2)

# Determine the best and worst campaigns based on cost per objective in the current period
best_campaign = merged_filtered.loc[merged_filtered['costo_por_objetivo_current'].idxmin()]
worst_campaign = merged_filtered.loc[merged_filtered['costo_por_objetivo_current'].idxmax()]

# Count initial and excluded campaigns
total_initial_campaigns = df['Campania'].nunique()
excluded_campaigns = total_initial_campaigns - merged_filtered['Campania'].nunique()

# Count exclusions by criteria
excluded_no_investment = total_initial_campaigns - df[df['inversion'] > 0]['Campania'].nunique()
excluded_no_events = total_initial_campaigns - df[df['evento_objetivo'] > 0]['Campania'].nunique()
excluded_incomplete_data = total_initial_campaigns - df[df['Fecha'].between(previous_period_start, current_period_end)]['Campania'].nunique()

# Prepare the summary of exclusions
exclusion_summary = {
    "total_initial_campaigns": total_initial_campaigns,
    "excluded_campaigns": excluded_campaigns,
    "excluded_no_investment": excluded_no_investment,
    "excluded_no_events": excluded_no_events,
    "excluded_incomplete_data": excluded_incomplete_data,
    "remaining_campaigns": merged_filtered['Campania'].nunique()
}

best_campaign, worst_campaign, exclusion_summary

(Campania                       BOOMIT_PEIG_EC_GADS_ADQUISI_ANDROID_(PRIMTARJ)...
 Plataforma                                                            Google Ads
 days_active_current                                                            7
 total_investment_current                                                 2841.49
 total_events_current                                                        1232
 days_active_previous                                                           7
 total_investment_previous                                                2994.09
 total_events_previous                                                        730
 costo_por_objetivo_current                                              2.306404
 costo_por_objetivo_previous                                             4.101493
 variacion_porcentual                                                      -43.77
 Name: 0, dtype: object,
 Campania                       BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(GAME